# Data Science Project Spring 2023

## 200+ Financial Indicators of US stocks (2014-2018)

### Yiwei Gong, Janice Herman, Alexander  Morawietz and Selina Waber

University of Zurich, Spring 2023

## Importing Packages

In [ ]:
import os 
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


## Loading the Data Set


We used the data set from Nicolas Carbone from the webpage [kaggle](https://www.kaggle.com/datasets/cnic92/200-financial-indicators-of-us-stocks-20142018). Each dataset contains over 200 financial indicators, that are found in the [10-K filings](https://www.investopedia.com/terms/1/10-k.asp#:~:text=Key%20Takeaways-,A%2010%2DK%20is%20a%20comprehensive%20report%20filed%20annually%20by,detailed%20than%20the%20annual%20report.) of publicly traded companies from the US between the years 2014 - 2018.

In [ ]:
project_directory = sys.path[0] ## get path of project directory
data_directory = os.path.join(project_directory, 'data')

years = [2014, 2015, 2016, 2017, 2018]

## Loading the yearly dataset into the array dfs
dfs = []
for year in years:
    df = pd.read_csv(os.path.join(data_directory, f'{year}_Financial_Data.csv'), sep=',')
    df['year'] = np.full(df.shape[0], str(year)) ## append column with the year respecitvely
    df['PRICE VAR [%]'] = df[f'{year +1} PRICE VAR [%]'] ## Adding variable of the same name for all df, e.g. '2016 PRICE VAR [%]' renamed to 'PRICE VAR [%]'
    df = df.drop(columns=[f'{year +1} PRICE VAR [%]']) # dropp year-specific variable name
    df.columns.values[0] = 'Stock Name' # name the first variable 
    dfs.append(df)
    
    
df = pd.concat(dfs) ## concat the diffrent dataframes
df.head()

## Some Explanation of Variables:

### Adding  `year` as a cathegorical variable

We added a column named `year` which contains the respecitve year.



### Handling the variable `Price VAR [%]`

The last column, `PRICE VAR [%]`, lists the percent price variation of each stock for the year. For example, if we consider the dataset 2015_Financial_Data.csv, we will have:

- 200+ financial indicators for the year 2015;
- percent price variation for the year 2016 (meaning from the first trading day on Jan 2016 to the last trading day on Dec 2016).

We renamed all the variables with the specific year in it, e.g.  `2016 PRICE VAR [%]` to `PRICE VAR [%]`. We dropped the old ones.Now we just have one variable `PRICE VAR [%]`. 


### The variable `class`

class lists a binary classification for each stock, where

- for each stock, if the PRICE VAR [%] value is positive, class = 1. From a trading perspective, the 1 identifies those stocks that an hypothetical trader should BUY at the start of the year and sell at the end of the year for a profit.
- for each stock, if the PRICE VAR [%] value is negative, class = 0. From a trading perspective, the 0 identifies those stocks that an hypothetical trader should NOT BUY, since their value will decrease, meaning a loss of capital.


The columns `PRICE VAR [%]` and `class` make possible to use the datasets for both classification and regression tasks:

- If the user wishes to train a machine learning model so that it learns to classify those stocks that in buy-worthy and not buy-worthy, it is possible to get the targets from the class column;
- If the user wishes to train a machine learning model so that it learns to predict the future value of a stock, it is possible to get the targets from the PRICE VAR [%] column.


### The variable  `Stock Name`

We named the first variable `Stock Name`since it has not been named in the original dataset.



## First Description of the Data

In [ ]:
df.info(verbose=True)

## Numerical and Catgorical Features/Variables

We are converting `Class`to a cathegorical variable.

In [ ]:
df['Class'] = df.Class.astype('object') ## object or catheogry?? whats the difference???

numCols = list(df.select_dtypes(exclude='object').columns)
print(f"There are {len(numCols)} numerical features:\n")

catCols = list(df.select_dtypes(include='object').columns)
print(f"There are {len(catCols)} categorical features:\n", catCols)

## Any Duplicates? 

No, there are no duplicates for rows but there are 20 duplicates for columns/ 10 each. Not same variable name but same data!

In [ ]:
print(f'Duplicates in Rows:', True in list(df.duplicated()))


In [ ]:
print(f'Duplicates in Columns:', True in list(df.T.duplicated().T))
print("Show the Duplicates:")
print(df.T[df.T.duplicated(keep=False)].T)

Our Duplicates are the following pairs:

- `ebitperRevenue` and `eBITperRevenu`
- `ebtperEBIT` and `eBTperEBIT`
- `niperEBT` and `nIperEBT`
- `returnOnAssets` and `Return on Tangible Assets`
- `returnOnCapitalEmployed` and `ROIC`
- `payablesTurnover` and `Payables Turnover`
- `inventoryTurnover` and `Inventory Turnover`
- `debtRatio` and `Debt to Assets`
- `debtEquityRatio` and `Debt to Equity`
- `cashFlowToDebtRatio` and `cashFlowCoverageRatios`

We will remove the first occurence of the duplicates respectively.

In [ ]:
shape_old=df.shape
df= df.T.drop_duplicates().T # remove duplicates!
print(f' Shape with duplicates:', shape_old) 
print(f' Shape after removal of duplicates:', df.shape) 

## Missing Values

There are a lot of missing values. 

In [ ]:
print(f'There are in total {df.isnull().sum().sum()} NAN in the dataframe')

In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
#df.replace(to_replace = 0, value = np.nan, inplace=true) ????? DA EVTL noch mehr mit NAN ersetzen!

In [ ]:
## Overview of all variables with missing values
df.isnull().mean().sort_values(ascending=False).plot.bar(figsize=(100,20))
plt.ylabel('Percentage of missing values')
plt.xlabel('Variables')
plt.title('Quantifying ALL missing data')


In [ ]:
most_nan = df.isnull().mean().sort_values(ascending=False)
most_nan = most_nan[most_nan > 0.3]

most_nan.plot.bar(figsize=(20,20))
plt.ylabel('Percentage of missing values')
plt.xlabel('Variables')
plt.title('Data with more than 30% missing')


In [ ]:
# Percentage of missing values for the variables
missing = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([missing, percent], axis=1, keys=['Nr. of missing values', 'Percent of Missing Values'])
missing_data.head(25)

In [ ]:
# Plot missing values 2.0
sns.heatmap(df.isna().transpose(), cmap="Blues", cbar_kws={'label': 'Missing Values'});

## Outliers cleaning

There are outliers/extreme values that are probably caused by mistypings. During our analysis of the data, we noticed that the values of NA and 0 were frequently used. We realized that 0 was used interchangeably with NA.  Also there are a lot of values that seem impossible. 

In [ ]:
## Z-Score: 

threshold = 3

test= df[['ebtperEBIT', 'returnOnCapitalEmployed']]
for col in test:
    z_score= stats.zscore(df[col], nan_policy='omit')
    outlier_indices = np.where(z_score > threshold)
    print(outlier_indices )
    

    

In [ ]:
# IQR
test= df[['ebtperEBIT', 'returnOnCapitalEmployed']]
for col in test:
    iqr= stats.iqr(df[col], nan_policy='omit')
    print(iqr)

#''''''
